# Este exemplo não utiliza redes convolucionais e está com problema durante o fit, que pára por um erro conhecido do Ipython. A solução é colocar verbose=0, mas o ruim é que não dá para acompanhar a evolução do erro durante o treinamento.


# Primeiro exemplo completo de rede neural usando Keras

Keras é uma biblioteca Python para desenvolver e avaliar modelos de aprendizagem profunda.

Ela é uma interface que facilita o uso das bibliotecas de computação numérica Theano e TensorFlow, que utilizam de GPU para obter alto desempenho nos algoritmos utilizados em redes convolucionais. Keras permite definir e treinar modelos de redes neurais em poucas linhas de código.

As etapas de programação da rede neural são as seguintes:

1. [Bibliotecas e inicialização](#1.-Bibliotecas-e-inicialização)
2. [Carregamento dos dados](#1.-Carregamento-dos-dados)
3. [Definição do modelo](#2.-Definição-do-modelo)
4. [Compilação](#3.-Compilação)
5. [Ajuste e treinamento](#4.-Ajuste-e-treinamento)
6. [Avaliação](#5.-Avaliação)
7. [Previsão](#6.-Previsão)

# 1. Bibliotecas e inicialização

Sempre que trabalhamos com algoritmos de aprendizado de máquina que usam um processo estocástico (por exemplo, números aleatórios), é recomendável definir uma sementre para inicializar o gerador de números aleatórios.

Dessa forma Se o código for executado novamente, os dados aleatórios serão exatamente o mesmo, gerando sempre o mesmo resultado. Isso é útil principalmente na fase de depuração do código.

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.data_utils import get_file
import numpy as np

np.random.seed(3) # inicialização da semente dos geradores aleatórios

# 1. Carregamento dos dados

Os dados deste exemplo são do conjunto de dados de diabetes dos índios Pima. Este é um conjunto de dados de aprendizagem de máquina padrão do repositório UCI Machine Learning. Ele descreve os dados do registro médico do paciente para os índios Pima e se eles tiveram um início de diabetes dentro de cinco anos.

Como tal, é um problema de classificação binária (início de diabetes como 1 ou não como 0). Todas as variáveis ​​de entrada que descrevem cada paciente são numéricas. Isso facilita o uso diretamente com redes neurais que esperam entrada numérica e valores de saída, e ideal para a nossa primeira rede neural em Keras.

O Keras tem uma boa função para carregar dados disponíveis na Internet: ``get_file``.

No código a seguir o conjunto de dados Pima Indian do repositório UCI Machine Learning é colocado no diretório local do Keras. É um arquivo .csv que pode ser lido facilmente pelo
numpy com a função ``loadtxt``. O conjunto de dados é composto de 846 amostras, oito atributos de entrada e um rótulo de saída (a última coluna). Uma vez carregado, podemos dividir o conjunto de dados em atributos de entrada (X) e o rótulo da classe de saída (Y).

In [13]:
path = 'pima-indians-diabetes.csv'
urlname = 'http://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data'
path = get_file(path, origin=urlname)
f = open(path, 'rb')
dataset = np.loadtxt(f, delimiter=",")
f.close()
print 'Leitura bem sucedida'
print 'Tipo dos dados: %s e forma dos dados %s' % (dataset.dtype, dataset.shape)

Leitura bem sucedida
Tipo dos dados: float64 e forma dos dados (768, 9)


In [12]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

X.shape, Y.shape, X.min(), X.max(), Y.min(), Y.max()

((768, 8), (768,), 0.0, 846.0, 0.0, 1.0)

# 2. Definição do modelo

Modelos em Keras são definidos como uma seqüência de camadas.

Criamos um modelo Seqüencial e adicionamos camadas uma de cada vez até ficarmos felizes com a nossa topologia de rede.

A primeira coisa a fazer é garantir que a camada de entrada tem o número certo de entradas. Isso pode ser especificado ao criar a primeira camada com o argumento input_dim e defini-lo como 8 para as 8 variáveis ​​de entrada.

Como sabemos o número de camadas e seus tipos?

Esta é uma pergunta muito difícil. Existem heurísticas que podemos usar e muitas vezes a melhor estrutura de rede é encontrada através de um processo de experimentação experimental. Geralmente, você precisa de uma rede grande o suficiente para capturar a estrutura do problema, se isso ajuda a todos.

Neste exemplo, usaremos uma estrutura de rede totalmente conectada com três camadas.

Camadas totalmente conectadas são definidas usando a classe Densa. Podemos especificar o número de neurônios na camada como o primeiro argumento, o método de inicialização como o segundo argumento como init e especificar a função de ativação usando o argumento de ativação.

Neste caso, inicializamos os pesos da rede para um pequeno número aleatório gerado a partir de uma distribuição uniforme ('uniforme'), neste caso entre 0 e 0,05, pois essa é a inicialização padrão uniforme em Keras. Outra alternativa tradicional seria "normal" para pequenos números aleatórios gerados a partir de uma distribuição gaussiana.

Usaremos a função de ativação do retificador ('relu') nas duas primeiras camadas e a função sigmóide na camada de saída. Costumava ser o caso que as funções de ativação sigmóide e tanh fossem preferidas para todas as camadas. Estes dias, melhor desempenho é alcançado usando a função de ativação de retificador. Usamos um sigmoide na camada de saída para garantir que nossa saída de rede esteja entre 0 e 1 e fácil de mapear para uma probabilidade de classe 1 ou para uma classificação rígida de qualquer classe com um limite padrão de 0,5.

Podemos juntar tudo juntando cada camada. A primeira camada tem 12 neurônios e espera 8 variáveis ​​de entrada. A segunda camada oculta tem 8 neurônios e, finalmente, a camada de saída tem 1 neurônio para prever a classe (início da diabetes ou não).

In [16]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))


# 3. Compilação

Agora que o modelo é definido, podemos compilá-lo.

A compilação do modelo utiliza as bibliotecas numéricas eficientes sob as tampas (o chamado backend), como Theano ou TensorFlow. O backend escolhe automaticamente a melhor maneira de representar a rede para treinamento e fazer previsões para serem executadas em seu hardware, como CPU ou GPU ou mesmo distribuídas.

Ao compilar, devemos especificar algumas propriedades adicionais necessárias ao treinar a rede. Lembre-se de treinar uma rede significa encontrar o melhor conjunto de pesos para fazer previsões para este problema.

Devemos especificar a função de perda a ser usada para avaliar um conjunto de pesos, o otimizador usado para pesquisar diferentes pesos para a rede e quaisquer métricas opcionais que gostaríamos de coletar e relatar durante o treinamento.

Neste caso, vamos usar a perda logarítmica, que para um problema de classificação binária é definida em Keras como "binary_crossentropy". Também usaremos o eficiente algoritmo de descida de gradiente "adam" por nenhuma outra razão que seja um padrão eficiente. Saiba mais sobre o algoritmo de otimização Adam no artigo "Adam: Um método para otimização estocástica".

Finalmente, por ser um problema de classificação, vamos coletar e relatar a precisão de classificação como a métrica.

In [17]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 4. Ajuste e treinamento

Definimos o nosso modelo e o compilamos pronto para computação eficiente.

Agora é hora de executar o modelo em alguns dados.

Nós podemos treinar ou ajustar nosso modelo em nossos dados carregados chamando a função fit () no modelo.

O processo de treinamento será executado para um número fixo de iterações através do dataset chamado epochs, que devemos especificar usando o argumento nb_epoch. Também podemos definir o número de instâncias que são avaliadas antes que uma atualização de peso na rede seja executada, chamada de tamanho de lote e definida usando o argumento batch_size.

Para este problema, vamos correr para um pequeno número de iterações (150) e usar um tamanho de lote relativamente pequeno de 10. Novamente, estes podem ser escolhidos experimentalmente por tentativa e erro.

In [25]:
# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)

# 5. Avaliação

Nós treinamos nossa rede neural em todo o conjunto de dados e podemos avaliar o desempenho da rede no mesmo conjunto de dados.

Isso só nos dará uma idéia de quão bem nós temos modelado o conjunto de dados (por exemplo, precisão de trem), mas nenhuma idéia de quão bem o algoritmo pode executar em novos dados. Fizemos isso por simplicidade, mas idealmente, você poderia separar seus dados em conjuntos de dados de treinamento e teste para treinamento e avaliação de seu modelo.

Você pode avaliar seu modelo em seu conjunto de dados de treinamento usando a função evaluate () em seu modelo e passar a mesma entrada e saída usada para treinar o modelo.

Isso gerará uma previsão para cada par de entrada e saída e colecionará partituras, incluindo a perda média e quaisquer métricas que você configurou, como exatidão.

In [26]:
# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

352/768 [============>.................] - ETA: 0sacc: 80.34%


# 6. Previsão

Fazer previsões é tão fácil quanto chamar model.predict (). Estamos usando uma função de ativação sigmóide na camada de saída, então as previsões estarão no intervalo entre 0 e 1. Podemos facilmente convertê-los em uma previsão binária nítida para esta tarefa de classificação, arredondando-os.

In [24]:
predictions = model.predict(X)
print(predictions)

[[  5.70532143e-01]
 [  8.00034925e-02]
 [  5.70532143e-01]
 [  1.26836821e-01]
 [  3.66572261e-01]
 [  2.18962297e-01]
 [  1.85624614e-01]
 [  5.47763407e-01]
 [  9.65664029e-01]
 [  5.27565889e-02]
 [  9.57360119e-02]
 [  5.70532143e-01]
 [  5.70532143e-01]
 [  7.49389172e-01]
 [  7.78657019e-01]
 [  3.87407035e-01]
 [  4.68728065e-01]
 [  3.05527598e-01]
 [  1.76451996e-01]
 [  2.99382389e-01]
 [  1.74271613e-01]
 [  3.88113707e-01]
 [  5.70532143e-01]
 [  3.15920115e-01]
 [  7.43505239e-01]
 [  6.11414850e-01]
 [  5.70532143e-01]
 [  5.54901809e-02]
 [  1.18394352e-01]
 [  2.02625021e-01]
 [  3.73778194e-01]
 [  6.50602758e-01]
 [  4.93336432e-02]
 [  1.49965296e-02]
 [  5.10442674e-01]
 [  5.60556650e-01]
 [  5.70532143e-01]
 [  4.04290110e-01]
 [  1.74906492e-01]
 [  4.92513597e-01]
 [  3.29905540e-01]
 [  5.70532143e-01]
 [  1.14950575e-01]
 [  7.21849024e-01]
 [  5.70532143e-01]
 [  5.70532143e-01]
 [  5.60989320e-01]
 [  6.31532744e-02]
 [  4.39956099e-01]
 [  1.45328701e-01]
